In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2023-09-25 12:50:05.473056: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-25 12:50:05.475630: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 12:50:05.523995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 12:50:05.525003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 12:50:06.307477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Data preprocessing

In [3]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [5]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [6]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

## Sequence helper

In [7]:
def generate_sequences(dataset, timesteps):
    dataset["x_train"], dataset["y_train"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_train"], columns=x.columns),
                dataset["y_train"],
            ],
            axis=1,
        ).values,
        timesteps,
    )

    dataset["x_test"], dataset["y_test"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_test"], columns=x.columns),
                dataset["y_test"],
            ],
            axis=1,
        ).values,
        timesteps,
    )
    return dataset

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [8]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    estimator_params=None,
    imputer_params=None,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [9]:
class GRU(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.GRU(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [10]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [11]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [12]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Dataframe structure to save the results

In [13]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "GRU",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [14]:
df = pd.read_csv("../../../../../data/processed/partner_ii/Type-I-II.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [15]:
df_copy = df.copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [16]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [17]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - GRU

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 41 samples each
    2. 90% train (369 samples each fold)
    3. 10% test (41 samples each fold)
<b>Total:</b> 100 models<br>

In [18]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"], axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        GRU,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        estimator_params=None,
        imputer_params={"strategy": "median"},
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-09-25 12:50:09.244064: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-25 12:50:09.244122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-09-25 12:50:09.244130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-09-25 12:50:09.244342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-09-25 12:50:09.244372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-09-25 12:50:09.244378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.407 (0.066)
MAE: 1.116 (0.058)
MAPE: 0.027 (0.001)
R2: 0.640 (0.034)


******
[TEST]
******
RMSE: 1.871 (0.763)
MAE: 1.281 (0.106)
MAPE: 0.031 (0.003)
R2: 0.251 (0.759)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.484 (0.183)
MAE: 1.200 (0.164)
MAPE: 0.028 (0.004)
R2: 0.596 (0.099)


******
[TEST]
******
RMSE: 3.449 (3.337)
MAE: 2.028 (0.890)
MAPE: 0.048 (0.021)
R2: -3.168 (10.409)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.220 (0.097)
MAE: 0.978 (0.084)
MAPE: 0.023 (0.002)
R2: 0.732 (0.041)


******
[TEST]
******
RMSE: 5.018 (6.527)
MAE: 2.630 (2.096)
MAPE: 0.063 (0.050)
R2: -10.505 (30.713)


Minutes Elapsed:  56.463948790232344


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 82 samples
    2. 50% train (41 samples each fold)
    3. 50% test (41 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            GRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 3.904 (0.382)
MAE: 3.017 (0.294)
MAPE: 0.072 (0.007)
R2: -2.893 (1.166)


******
[TEST]
******
RMSE: 12.501 (6.986)
MAE: 8.578 (2.407)
MAPE: 0.207 (0.064)
R2: -60.418 (99.177)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.579 (0.302)
MAE: 1.296 (0.258)
MAPE: 0.031 (0.006)
R2: 0.350 (0.250)


******
[TEST]
******
RMSE: 22.928 (24.367)
MAE: 15.314 (12.142)
MAPE: 0.364 (0.286)
R2: -337.919 (727.024)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.352 (0.153)
MAE: 1.078 (0.121)
MAPE: 0.026 (0.003)
R2: 0.532 (0.108)


******
[TEST]
******
RMSE: 12.769 (11.508)
MAE: 10.336 (8.686)
MAPE: 0.246 (0.204)
R2: -99.484 (163.746)


Minutes Elapsed:  19.47337135473887


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 40, 77, 114, 152, 189, 226, 263, 301, 338 samples each fold
    2. Test: 37 samples each fold
<b>Total:</b> 10 models<br>

In [20]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            GRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.030 (1.079)
MAE: 1.600 (0.848)
MAPE: 0.038 (0.021)
R2: -0.555 (1.942)


******
[TEST]
******
RMSE: 5.405 (3.332)
MAE: 3.812 (2.969)
MAPE: 0.090 (0.069)
R2: -9.008 (10.421)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.683 (0.362)
MAE: 1.398 (0.346)
MAPE: 0.033 (0.008)
R2: 0.223 (0.340)


******
[TEST]
******
RMSE: 6.856 (4.415)
MAE: 5.198 (3.764)
MAPE: 0.122 (0.087)
R2: -15.721 (22.241)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.648 (0.460)
MAE: 1.370 (0.426)
MAPE: 0.032 (0.010)
R2: 0.221 (0.517)


******
[TEST]
******
RMSE: 5.464 (4.750)
MAE: 4.130 (3.881)
MAPE: 0.098 (0.090)
R2: -11.586 (23.083)


Minutes Elapsed:  58.09055891434352


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 328
    2. Test: 82
<b>Total:</b> 1 model<br>

In [21]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)

        # Sequence Splitting
        data_train = pd.concat(
            [
                pd.DataFrame(x_train, columns=x.columns),
                y_train.reset_index(drop=True),
            ],
            axis=1,
        ).values
        data_test = pd.concat(
            [
                pd.DataFrame(x_test, columns=x.columns),
                y_test.reset_index(drop=True),
            ],
            axis=1,
        ).values

        x_train, y_train = split_sequences(data_train, timesteps)
        x_test, y_test = split_sequences(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", GRU())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.366 (0.073)
MAE: 1.091 (0.069)
MAPE: 0.026 (0.002)
R2: 0.558 (0.048)


******
[TEST]
******
RMSE: 1.740 (0.052)
MAE: 1.364 (0.058)
MAPE: 0.035 (0.001)
R2: 0.384 (0.037)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.254 (0.119)
MAE: 1.014 (0.098)
MAPE: 0.024 (0.002)
R2: 0.625 (0.073)


******
[TEST]
******
RMSE: 2.686 (0.313)
MAE: 2.193 (0.332)
MAPE: 0.056 (0.008)
R2: -0.596 (0.378)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.168 (0.027)
MAE: 0.944 (0.027)
MAPE: 0.022 (0.001)
R2: 0.680 (0.015)


******
[TEST]
******
RMSE: 2.803 (0.432)
MAE: 2.275 (0.374)
MAPE: 0.059 (0.010)
R2: -0.806 (0.534)


Minutes Elapsed:  17.743607306480406


<IPython.core.display.Javascript object>

In [22]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation  \
                                                                            
0   Chemical + Mineralogical + Physical   GRU  Blocking Time Series Split   
1   Chemical + Mineralogical + Physical   GRU  Blocking Time Series Split   
2   Chemical + Mineralogical + Physical   GRU  Blocking Time Series Split   
3   Chemical + Mineralogical + Physical   GRU           Out of time Split   
4   Chemical + Mineralogical + Physical   GRU           Out of time Split   
5   Chemical + Mineralogical + Physical   GRU           Out of time Split   
6   Chemical + Mineralogical + Physical   GRU              Repeated KFold   
7   Chemical + Mineralogical + Physical   GRU              Repeated KFold   
8   Chemical + Mineralogical + Physical   GRU              Repeated KFold   
9   Chemical + Mineralogical + Physical   GRU           Time Series Split   
10  Chemical + Mineralogical + Physical   GRU           Time Series Split   
11  Chemical + Mineralogical + Physical   GRU           Time Series Split   

   Timesteps  RMSE Test              MAE Test            MAPE Test            \
                   mean        std       mean        std      mean       std   
0          1  12.500833   6.986461   8.578019   2.406649  0.206778  0.063834   
1          7  22.928127  24.366939  15.313598  12.141771  0.364425  0.285513   
2         14  12.769254  11.507699  10.335513   8.685682  0.245739  0.204135   
3          1   1.740042   0.051509   1.363714   0.058073  0.034822  0.001468   
4          7   2.686374   0.312806   2.192685   0.331854  0.056260  0.008290   
5         14   2.803167   0.431520   2.274853   0.373990  0.059156  0.009923   
6          1   1.871193   0.762854   1.281190   0.105727  0.030607  0.002599   
7          7   3.448785   3.337149   2.028244   0.890313  0.048289  0.021086   
8         14   5.018474   6.527006   2.630053   2.096182  0.063011  0.050141   
9          1   5.405102   3.332063   3.811724   2.968939  0.090168  0.068985   
10         7   6.855958   4.414652   5.197689   3.763509  0.122449  0.086944   
11        14   5.463988   4.749531   4.130298   3.881045  0.097788  0.090158   

       R2 Test              
          mean         std  
0   -60.417798   99.177111  
1  -337.918958  727.024296  
2   -99.483831  163.745764  
3     0.384199    0.036805  
4    -0.596047    0.378019  
5    -0.805899    0.534371  
6     0.250564    0.759487  
7    -3.168478   10.409476  
8   -10.504762   30.712547  
9    -9.007743   10.420522  
10  -15.721010   22.240791  
11  -11.586104   23.082924

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [23]:
path = "../../../../../reports/results/local_models/partner_ii/type-I-II/full/"
filename = "gru_results_full_1.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [24]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_ii/type-I-II/grouped/"
filename = "gru_results_grouped_1.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>